In [30]:
import numpy as np
import fiona
import pprint
from madmex.models import PredictObject
from madmex.models import PredictClassification, Region, ValidClassification
from madmex.util import chunk
import json
from shapely.geometry import mapping
from madmex.util.spatial import geometry_transform
from shapely.geometry import shape
from dask.distributed import Client
import os
from operator import itemgetter
from django.contrib.gis.geos.geometry import GEOSGeometry
from fiona.crs import to_string

In [2]:
name_predict = 'lc_rf_s1_2_20m_resampled_10m_Jalisco_recipe_2017'

In [3]:
qs_ids = PredictClassification.objects.filter(name=name_predict).distinct('predict_object_id')

In [4]:
len(qs_ids)

56

In [5]:
list_ids = [x.predict_object_id for x in qs_ids]

In [6]:
region = 'Jalisco'

In [7]:
region_geom = Region.objects.get(name=region).the_geom
region_geojson = region_geom.geojson
geometry = json.loads(region_geojson)
proj4='+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'
geometry_region_proj = geometry_transform(geometry,proj4)

In [8]:
list_ids[0]

169

In [9]:
len(list_ids)

56

In [10]:
validation_set='validation_Jalisco'

In [25]:
list_qs = []
list_seg_paths = []
proj4_out = '+proj=longlat'
proj4_in = '+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'
validation_set='validation_Jalisco'

In [26]:
for id in list_ids:
    seg = PredictObject.objects.filter(id=id)
    path = seg[0].path
    poly = seg[0].the_geom
    poly_geojson = poly.geojson
    geometry = json.loads(poly_geojson)
    geometry_proj = geometry_transform(geometry,proj4_out,crs_in=proj4_in)
    poly_proj = GEOSGeometry(json.dumps(geometry_proj))
    query_set = ValidClassification.objects.filter(valid_object__the_geom__contained=poly_proj,
                                               valid_set=validation_set).prefetch_related('valid_object', 'valid_tag')
    list_qs.append(query_set)
    list_seg_paths.append(path)

In [13]:
list_qs

[<QuerySet [<ValidClassification: ValidClassification object (268)>, <ValidClassification: ValidClassification object (367)>, <ValidClassification: ValidClassification object (385)>]>,
 <QuerySet []>,
 <QuerySet []>,
 <QuerySet [<ValidClassification: ValidClassification object (301)>, <ValidClassification: ValidClassification object (302)>, <ValidClassification: ValidClassification object (460)>, <ValidClassification: ValidClassification object (461)>, <ValidClassification: ValidClassification object (473)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (468)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (519)>, <ValidClassification: ValidClassification object (520)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (324)>, <ValidClassification: ValidClassification object (327)>, <ValidClassification: ValidClassification object (328)>, <ValidClassification: ValidClassification object (337)>, <ValidClassification: ValidClassification obj

In [17]:
qs_dc_tile = list_qs[0]

In [18]:
qs_dc_tile

<QuerySet [<ValidClassification: ValidClassification object (268)>, <ValidClassification: ValidClassification object (367)>, <ValidClassification: ValidClassification object (385)>]>

In [28]:
seg_path_s3 = list_seg_paths[0]

In [20]:
qs_element_dc_tile = qs_dc_tile[0]

In [24]:
qs_element_dc_tile.valid_tag.id

8

In [42]:
def valid_object_to_feature(x, crs):
    attr = {'class': x.valid_tag.numeric_code}
    if crs is None:
        geometry = json.loads(x.valid_object.the_geom.geojson)
    else:
        geometry = json.loads(x.valid_object.the_geom.transform(crs, clone=True).geojson)
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": attr
    }
    return feature

In [45]:
with fiona.open(seg_path_s3) as src:
    crs = to_string(src.crs)
    fc_qs = [valid_object_to_feature(x, crs) for x in qs_dc_tile]

In [46]:
fc_qs[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[2425342.90970262, 833105.8176630371],
    [2425377.7764724013, 833105.1778357779],
    [2425377.685318055, 833100.1670191728],
    [2425387.6472516027, 833099.9842140923],
    [2425387.5560979685, 833094.9733974674],
    [2425392.537064822, 833094.8819954653],
    [2425392.445911615, 833089.8711786619],
    [2425397.4268786185, 833089.7797770762],
    [2425397.335725839, 833084.7689601114],
    [2425402.3166929884, 833084.6775589335],
    [2425402.134388419, 833074.6559243695],
    [2425407.115356006, 833074.5645237326],
    [2425407.024204218, 833069.5537061333],
    [2425412.0051719537, 833069.4623059129],
    [2425411.822869368, 833059.44067006],
    [2425406.8419010574, 833059.5320700057],
    [2425406.5684473533, 833044.49961352],
    [2425411.5494165304, 833044.4082139963],
    [2425411.4582658606, 833039.3973946947],
    [2425416.4392351853, 833039.3059955874],
    [2425416.2569348384, 833029.284356344],
  